<h1 align="center"><font color="yellow">Como criar uma classe de memória personalizada</font></h1>

<font color="yellow">Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro</font>

Embora existam alguns tipos predefinidos de memória no `LangChain`, é altamente possível que você queira adicionar `seu próprio tipo de memória ideal para sua aplicação`. Este Notebook cobre como fazer isso.

Para este notebook, adicionaremos um tipo de `memória customizada` a `ConversationChain`. Para adicionar uma classe de memória personalizada, precisamos importar a `classe de memória base` e criar uma `subclasse` dela.

In [1]:
# Isto é quando usas o arquivo .env:
from dotenv import load_dotenv
import os
print('Carregando a minha chave Key: ', load_dotenv())
Eddy_API_KEY_OpenAI = os.environ['OPENAI_API_KEY'] 

Carregando a minha chave Key:  True


In [2]:
from langchain import OpenAI, ConversationChain
from langchain.schema import BaseMemory
from pydantic import BaseModel
from typing import List, Dict, Any


Neste exemplo, escreveremos uma `classe de memória personalizada` que usa `spaCy` para extrair Entidades e salvar informações sobre elas em uma `tabela de hash` simples. Então, durante a conversa, vamos olhar para o `texto de entrada`, extrair quaisquer entidades e colocar qualquer informação sobre elas no contexto.

* Observe que esta implementação é bastante simples e frágil e provavelmente não é útil em uma configuração de produção. Sua finalidade é mostrar que você pode adicionar implementações de memória personalizadas.

Para isso, precisaremos do `spaCy`. Seguidamente fazemos as instalação:

In [3]:
#%pip install spacy
#%python -m spacy download en_core_web_lg
# Nós usaremos um modelo pré-treinado em Português:
#%python -m spacy download pt_core_news_sm

In [3]:
import spacy
nlp = spacy.load('pt_core_news_sm')


In [4]:
class SpacyEntityMemory(BaseMemory, BaseModel):
    """Classe de memória para armazenar informações sobre Entidades."""

    # Definir dicionário para armazenar informações sobre entidades.
    entities: dict = {}
    # Definir chave para passar informações sobre entidades no prompt.
    memory_key: str = "entities"
        
    def clear(self):
        self.entities = {}

    @property
    def memory_variables(self) -> List[str]:
        """Defina as variáveis que estamos fornecendo ao prompt."""
        return [self.memory_key]

    def load_memory_variables(self, inputs: Dict[str, Any]) -> Dict[str, str]:
        """Carregue as variáveis de memória, neste caso a chave da Entidade."""
        # Obtenha o texto de entrada e execute o spaCy 
        doc = nlp(inputs[list(inputs.keys())[0]])
        # Extraia informações conhecidas sobre Entidades, se existirem.
        entities = [self.entities[str(ent)] for ent in doc.ents if str(ent) in self.entities]
        # Retorne informações combinadas sobre entidades para coloque em contexto.
        return {self.memory_key: "\n".join(entities)}

    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        """Salve o contexto desta conversa no buffer."""
        # Obtenha o texto de entrada e execute através do spaCy
        text = inputs[list(inputs.keys())[0]]
        doc = nlp(text)
        # Para cada entidade mencionada, salve essa informação no dicionário.
        for ent in doc.ents:
            ent_str = str(ent)
            if ent_str in self.entities:
                self.entities[ent_str] += f"\n{text}"
            else:
                self.entities[ent_str] = text


<font color="orange">Agora definimos um prompt que recebe informações sobre Entidades, bem como entrada do `usuário`.</font>

In [5]:
from langchain.prompts.prompt import PromptTemplate

template = """O que se segue é uma conversa amigável entre um humano e uma IA. A IA é falante e fornece muitos
detalhes específicos de seu contexto. Se a IA não souber a resposta para uma pergunta, ela diz com sinceridade
que não sabe. Você recebe informações sobre as entidades mencionadas pelo Humano, se for o caso.

Relevant entity information:
{entities}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(
    input_variables=["entities", "input"],
    template=template
                       )



<font color="orange">E agora juntamos tudo!</font>

In [6]:
llm = OpenAI(temperature=0)

conversation = ConversationChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=SpacyEntityMemory()
                                )


<font color="orange">No `primeiro exemplo:` sem nenhum conhecimento prévio sobre `Eddy Giusepe`, a seção `“Informações relevantes da Entidade”` está vazia.</font>

In [7]:
conversation.predict(input="Eu sou Eddy Giusepe e estudei na UFES o curso de Física teórica e hoje em dia sou Cientista de Dados.")





> Entering new ConversationChain chain...
Prompt after formatting:
O que se segue é uma conversa amigável entre um humano e uma IA. A IA é falante e fornece muitos
detalhes específicos de seu contexto. Se a IA não souber a resposta para uma pergunta, ela diz com sinceridade
que não sabe. Você recebe informações sobre as entidades mencionadas pelo Humano, se for o caso.

Relevant entity information:


Conversation:
Human: Eu sou Eddy Giusepe e estudei na UFES o curso de Física teórica e hoje em dia sou Cientista de Dados.
AI:

> Finished chain.


' Olá Eddy Giusepe! É um prazer conhecê-lo. Então, você estudou Física Teórica na UFES e agora é Cientista de Dados? Isso é muito interessante! Você gosta do que faz?'

<font color="orange">Agora, no `segundo exemplo:` podemos ver que ele extrai informações sobre Eddy Giusepe.</font>

In [8]:
conversation.predict(input="Sou apaxionado pelo que faço sim. Aqui, preciso de informações sobre onde posso fazer uma matrícula para aprimorar meus conhecimentos?")




> Entering new ConversationChain chain...
Prompt after formatting:
O que se segue é uma conversa amigável entre um humano e uma IA. A IA é falante e fornece muitos
detalhes específicos de seu contexto. Se a IA não souber a resposta para uma pergunta, ela diz com sinceridade
que não sabe. Você recebe informações sobre as entidades mencionadas pelo Humano, se for o caso.

Relevant entity information:


Conversation:
Human: Sou apaxionado pelo que faço sim. Aqui, preciso de informações sobre onde posso fazer uma matrícula para aprimorar meus conhecimentos?
AI:

> Finished chain.


' Claro! Existem muitos cursos online que você pode fazer para aprimorar seus conhecimentos. Alguns deles são oferecidos por universidades renomadas, como a Universidade de Harvard, a Universidade de Stanford e a Universidade de Oxford. Você também pode procurar por cursos específicos em sites como o Coursera, o Udemy e o edX. Além disso, existem muitos cursos gratuitos disponíveis em plataformas como o Khan Academy.'

In [9]:
conversation.memory

SpacyEntityMemory(entities={'Eddy Giusepe': 'Eu sou Eddy Giusepe e estudei na UFES o curso de Física teórica e hoje em dia sou Cientista de Dados.', 'UFES': 'Eu sou Eddy Giusepe e estudei na UFES o curso de Física teórica e hoje em dia sou Cientista de Dados.', 'Física': 'Eu sou Eddy Giusepe e estudei na UFES o curso de Física teórica e hoje em dia sou Cientista de Dados.', 'Cientista de Dados': 'Eu sou Eddy Giusepe e estudei na UFES o curso de Física teórica e hoje em dia sou Cientista de Dados.'}, memory_key='entities')

<font color="red">NOTA:</font>

Esta implementação é apenas um exemplo e não é recomendado para um ambiente de produção.